####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import col, to_date, when

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 5cbd29ae-54d0-44c7-96f1-d3607cc34429
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session 5cbd29ae-54d0-44c7-96f1-d3607cc34429 to get into ready status...
Session 5cbd29ae-54d0-44c7-96f1-d3607cc34429 has been created.



#### JDBC CONNECTION


In [2]:
rds_jdbc_url = "jdbc:mysql://apartment-db.cl4s228oi3rd.eu-west-1.rds.amazonaws.com:3306/apartmentDB"

In [3]:
common_options = {
    "url": rds_jdbc_url,
    "user": "admin",
    "password": "Cha_rles123", 
    "customJdbcDriverS3Path": "s3://rds-s3-bkttt/connector/mysql-connector-j-8.4.0.jar",
    "customJdbcDriverClassName": "com.mysql.cj.jdbc.Driver"
}

In [4]:
tables = ["apartment", "apartment_attributes", "user_viewings", "bookings"]
s3_base_path = "s3://rds-s3-bktt/loads_from_rds/"

In [ ]:
for table in tables:
    print(f"\n🔹 Extracting data from table: {table}")

    # Extract data from MySQL table into a Spark DataFrame
    options = common_options.copy()
    options["dbtable"] = table

    spark_df = spark.read.format("jdbc").options(**options).load()

    # **DEBUG: Print schema and row count**
    print(f"Schema for {table}:")
    spark_df.printSchema()
    
    row_count = spark_df.count()
    print(f"Count for {table}: {row_count}")



    # Convert Spark DataFrame to DynamicFrame
    dynamic_frame = DynamicFrame.fromDF(spark_df, glueContext, "dynamic_df")
    print(f"DynamicFrame Schema for {table}: {dynamic_frame.schema()}")

    # Define S3 path
    s3_path = f"{s3_base_path}{table}/"
    print(f"🔹 Writing {row_count} rows to {s3_path}")

    # Write DynamicFrame to S3 in Parquet format
    glueContext.write_dynamic_frame.from_options(
        frame=dynamic_frame,
        connection_type="s3",
        connection_options={"path": s3_path},
        format="parquet"
    )



🔹 Extracting data from table: apartment
Schema for apartment:
root
 |-- id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- source: string (nullable = true)
 |-- price: decimal(10,2) (nullable = true)
 |-- currency: string (nullable = true)
 |-- listing_created_on: timestamp (nullable = true)
 |-- is_active: boolean (nullable = true)
 |-- last_modified_timestamp: timestamp (nullable = true)

Count for apartment: 200000
DynamicFrame Schema for apartment: StructType([Field(id, LongType({}), {}),Field(title, StringType({}), {}),Field(source, StringType({}), {}),Field(price, DecimalType(10, 2, {}), {}),Field(currency, StringType({}), {}),Field(listing_created_on, TimestampType({}), {}),Field(is_active, BooleanType({}), {}),Field(last_modified_timestamp, TimestampType({}), {})], {})
🔹 Writing 200000 rows to s3://rds-s3-bktt/loads_from_rds/apartment/

🔹 Extracting data from table: apartment_attributes
Schema for apartment_attributes:
root
 |-- apartment_id: long (nullable =

In [8]:
s3_df = spark.read.parquet(s3_path)
s3_df.show(5)  # Show sample rows
s3_df.printSchema()  # Validate column names and types


+----------+-------+------------+-------------------+-------------------+-------------------+-----------+--------+--------------+
|booking_id|user_id|apartment_id|       booking_date|       checkin_date|      checkout_date|total_price|currency|booking_status|
+----------+-------+------------+-------------------+-------------------+-------------------+-----------+--------+--------------+
|         1|   2920|      130940|2022-11-11 00:00:00|2024-09-07 00:00:00|2023-06-30 00:00:00|    2923.67|     USD|     confirmed|
|         2|   2788|      191497|2024-05-17 00:00:00|2024-07-07 00:00:00|2021-09-06 00:00:00|    4645.29|     USD|       pending|
|         3|   9285|      139117|2022-11-26 00:00:00|2025-02-12 00:00:00|2023-04-07 00:00:00|    1683.99|     USD|      canceled|
|         4|   8161|      118131|2020-07-09 00:00:00|2024-10-25 00:00:00|2021-02-08 00:00:00|    1870.62|     USD|     confirmed|
|         5|   6733|       43470|2024-05-08 00:00:00|2022-01-31 00:00:00|2024-10-29 00:00: